<a href="https://colab.research.google.com/github/francji1/01NAEX/blob/main/code/01NAEX_Exercise_01_Python_student_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01NAEX - Exercise 01
Data and exercises come from D.C. Montgomery: Design and Analysis of Experiment


## Setup

In [ ]:
!pip install rpy2

In [ ]:
%load_ext rpy2.ipython

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm, t, f, shapiro


# Example from the Lecture

In [ ]:
# Data arrays
y1 = np.array([16.85,16.40,17.21,16.35,16.52,17.04,16.96,17.15,16.59,16.57])  # Modified Mortar
y2 = np.array([16.62,16.75,17.37,17.12,16.98,16.87,17.34,17.02,17.08,17.27])  # Unmodified Mortar

# Sample variances
s1_squared = np.var(y1, ddof=1)
s2_squared = np.var(y2, ddof=1)

# Degrees of freedom
dfn = len(y1) - 1  # Degrees of freedom numerator
dfd = len(y2) - 1  # Degrees of freedom denominator

# F-test statistic
F = s1_squared / s2_squared

# Two-tailed p-value
p_value = 2 * min(f.cdf(F, dfn, dfd), 1 - f.cdf(F, dfn, dfd))

print('F-statistic:', F)
print('Degrees of freedom:', dfn, 'and', dfd)
print('p-value:', p_value)

In [ ]:
# Independent two-sample t-test (equal variances)
t_statistic, p_value = stats.ttest_ind(y1, y2, equal_var=True)

print(f't-statistic: {t_statistic}')
print(f'p-value: {p_value}')

In [ ]:
# Welch's t-test (unequal variances)
t_stat, p_value = stats.ttest_ind(y1, y2, equal_var=False)

print('Welch\'s t-statistic:', t_stat)
print('p-value:', p_value)

In [ ]:
# 1. Two-sample t-test assuming equal variances (var.equal = TRUE in R)
t_stat_equal_var, p_value_equal_var = stats.ttest_ind(y1, y2, equal_var=True)

# Calculate confidence interval for equal variance
n1, n2 = len(y1), len(y2)
mean_diff = np.mean(y1) - np.mean(y2)
pooled_std = np.sqrt(((n1 - 1) * np.var(y1, ddof=1) + (n2 - 1) * np.var(y2, ddof=1)) / (n1 + n2 - 2))
se_pooled = pooled_std * np.sqrt(1/n1 + 1/n2)
conf_interval_equal_var = stats.t.interval(0.95, df=n1 + n2 - 2, loc=mean_diff, scale=se_pooled)

# 2. Welch's t-test (var.equal = FALSE in R)
t_stat_unequal_var, p_value_unequal_var = stats.ttest_ind(y1, y2, equal_var=False)
df_unequal_var = ((np.var(y1, ddof=1)/n1 + np.var(y2, ddof=1)/n2)**2) / \
                 ((np.var(y1, ddof=1)/n1)**2/(n1-1) + (np.var(y2, ddof=1)/n2)**2/(n2-1))

# Calculate confidence interval for unequal variance
se_unequal = np.sqrt(np.var(y1, ddof=1)/n1 + np.var(y2, ddof=1)/n2)
conf_interval_unequal_var = stats.t.interval(0.95, df=df_unequal_var, loc=mean_diff, scale=se_unequal)

# Results for t-test assuming equal variances
print("Two-Sample T-Test Assuming Equal Variances")
print(f"t-statistic: {t_stat_equal_var}")
print(f"p-value: {p_value_equal_var}")
print(f"95% confidence interval: {conf_interval_equal_var}")
print(f"Mean of y1: {np.mean(y1)}, Mean of y2: {np.mean(y2)}")
print()

# Results for Welch's t-test (unequal variances)
print("Welch's Two-Sample T-Test (Assuming Unequal Variances)")
print(f"t-statistic: {t_stat_unequal_var}")
print(f"p-value: {p_value_unequal_var}")
print(f"Degrees of freedom: {df_unequal_var}")
print(f"95% confidence interval: {conf_interval_unequal_var}")
print(f"Mean of y1: {np.mean(y1)}, Mean of y2: {np.mean(y2)}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Kernel Density Plot
sns.kdeplot(y1, fill=True, label='Modified Mortar')
sns.kdeplot(y2, fill=True, label='Unmodified Mortar')
plt.title('Kernel Density Estimation of Mortar Data')
plt.xlabel('Tension Bond Strength')
plt.legend()
plt.show()

In [ ]:
# QQ-Plot for y1
plt.figure()
stats.probplot(y1, dist="norm", plot=plt)
plt.title('Normal QQ-Plot for Modified Mortar')
plt.show()

# QQ-Plot for y2
plt.figure()
stats.probplot(y2, dist="norm", plot=plt)
plt.title('Normal QQ-Plot for Unmodified Mortar')
plt.show()

In [ ]:
# Shapiro-Wilk test for y1
statistic_y1, p_value_y1 = shapiro(y1)
print(f'Shapiro-Wilk Test for y1: Statistic={statistic_y1}, p-value={p_value_y1}')

# Shapiro-Wilk test for y2
statistic_y2, p_value_y2 = shapiro(y2)
print(f'Shapiro-Wilk Test for y2: Statistic={statistic_y2}, p-value={p_value_y2}')

In [ ]:
from statsmodels.stats.power import TTestIndPower

# Parameters
effect_size = (np.mean(y1) - np.mean(y2)) / np.sqrt((s1_squared + s2_squared) / 2)
alpha = 0.05
power = 0.95

# Create an instance of the power analysis class
analysis = TTestIndPower()

# Calculate required sample size
sample_size = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, alternative='two-sided')
print(f'Required sample size per group: {np.ceil(sample_size)}')

# Calculate power of the test with n=10
actual_power = analysis.power(effect_size=effect_size, nobs1=10, alpha=alpha, alternative='two-sided')
print(f'Power of the test with n=10 per group: {actual_power}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.power import TTestIndPower

# Parameters
alpha = 0.05
power = 0.80
sd = 0.284  # Standard deviation
effect_sizes = np.array([0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6])

# Calculate sample sizes
analysis = TTestIndPower()
sample_sizes = []
for delta in effect_sizes:
    effect_size = delta / sd
    n = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, alternative='two-sided')
    sample_sizes.append(n)

# Plotting
plt.plot(effect_sizes, sample_sizes, marker='o')
plt.xlabel('Effect Size')
plt.ylabel('Sample Size per Group')
plt.title('Sample Size vs. Effect Size')
plt.grid(True)
plt.show()


## Assigment:

* Run and familiarize with Python.
* Solve following problems from Montgomery - Design and Analysis of Experiments.


### Exercises 2.20

The shelf life of a carbonated beverage is of interest. Ten bottles are randomly
selected and tested, and the following results are obtained:
| Days |     |
|------|-----|
| 108  | 138 |
| 124  | 163 |
| 124  | 159 |
| 106  | 134 |
| 115  | 139 |


* We would like to demonstrate that the mean shelf life exceeds 120 days.
Set up appropriate hypotheses for investigating this claim.
* Test these hypotheses using significant level $\alpha = 0.01$. Find the P-value
for the test. What are your conclusions?
* Construct a 99 percent confidence interval on the mean shelf life.
* Can shelf life be adequately described or modeled by a normal distribution? What effect would a violation of this assumption have on the test procedure you used in solving previous questions?

In [ ]:
# Read the data from the URL
url_20 = "https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex02_20.csv"
df20 = pd.read_csv(url_20, sep=";")

# Display the first few rows of the dataframe
df20.head(10)


SOLUTION:

hypotheses setup: H0: μ = 120, H1: μ > 120

In [ ]:
mean = df20['Days'].mean()
std = df20['Days'].std()

df = len(df20['Days'])-1

# test statistic
t_stat = (mean - 120) / (std / np.sqrt(10))

#to reject the null hypothesis, the estimated mean has to be greater than 120 (so the statistic > 0) and the p-value of the test statistic needs to be lower than 0.01 (we do only one-sided test)
p_val = t.sf(np.abs(t_stat), df)
print(f't_stat: {t_stat}, p_val: {p_val}')


Suppose normality of the distribution of shelf life, so $shelf \text{ }life ∼ N(μ , σ)$. In gerenal, the unbiased estimate of sample mean of random variable $X$ is $\hat{\mu} = \frac{1}{n}\sum_{i = 1}^{n}X_i$, where $X_i$ $iid$ replications of $X$. In our case,  $X \sim N(μ , σ)$, thus, from the reproductive property of normal distribution we obtain $\hat{\mu} \sim N(\mu, \frac{\sigma}{\sqrt{n}})$. Of course, when $\sigma$ is not known, one needs to use the unbiased estimate $\hat{\sigma}$.



On the other hand, one might argue that shelf life is non-negative, thus cannot be modelled as a random variable with normal distribution. As long as shelf life is $\mathcal{L}_2$ random variable and the replications are $iid$, we can rely on the central limit theorem, according to which $\hat{\mu}$ is at least asymptotically normal. One thing to consider is that 10 observations might not be enough to guarantee that the distribution of $\hat{\mu}$ is sufficiently close to normal.

In [ ]:
quantile_995 = norm.ppf(0.995, loc=0, scale=1)

print(f'the 99% [{mean - std / np.sqrt(10)*quantile_995}, {mean + std / np.sqrt(10)*quantile_995}]')

### Exercise 2-21

In semiconductor manufacturing, wet chemical etching is often used to remove silicon from the backs of wafers prior to metallization. The **etch rate** is an important characteristic of this process. Two different etching solutions are being evaluated. Eight randomly selected wafers have been etched in each solution and the observed etch rates (in mils/min) are shown below:

| Solution 1 | Solution 2 |
|------------|------------|
|  9.9       | 10.2       |
|  9.4       | 10.0       |
| 10.0       | 10.7       |
| 10.3       | 10.5       |
| 10.6       | 10.6       |
| 10.3       | 10.2       |
|  9.3       | 10.4       |
|  9.8       | 10.3       |

**(a)** Do the data indicate that the claim that both solutions have the same mean etch rate is valid? Use α = 0.05 and assume equal variances.  

**(b)** Find a 95% confidence interval on the difference in mean etch rates.  

**(c)** Use normal probability plots to investigate the adequacy of the assumptions of normality and equal variances.  

**(d)** Compute the power of the test in part (a). If the variance and means corresponds to estimations based on enclosed data, how many measurements per group would be required to achieve power greater than 0.9 to detect a difference of Δ = 0.3 at significance level α = 0.05?


In [ ]:
# Read the data from the URL
url_20 = "https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex02_21.csv"
df20 = pd.read_csv(url_20, sep=";")

# Display the first few rows of the dataframe
df20.head()

Solution:

In [ ]:
mean_1 = df20['Solution1'].mean()
mean_2 = df20['Solution2'].mean()
std_1 = df20['Solution1'].std()
std_2 = df20['Solution2'].std()

pooled_std = np.sqrt(((len(df20['Solution1'])-1) * std_1**2 + (len(df20['Solution2'])-1) * std_2**2) / (len(df20['Solution1'])+len(df20['Solution2'])-2))

# test statistic
t_stat = (mean_1 - mean_2) / (pooled_std * np.sqrt(1/len(df20['Solution1']) + 1/len(df20['Solution2'])))

# we can answer question a) and b) at the same time since rejecting null hypothesis on 0.05 confidence level is equivalent to the fact that 0 does not lie within the 95% confidence interval
b_lower = mean_1 - mean_2 - t.ppf(0.975, len(df20['Solution1'])+len(df20['Solution2'])-2) * pooled_std * np.sqrt(1/len(df20['Solution1']) + 1/len(df20['Solution2']))
b_upper = mean_1 - mean_2 + t.ppf(0.975, len(df20['Solution1'])+len(df20['Solution2'])-2) * pooled_std * np.sqrt(1/len(df20['Solution1']) + 1/len(df20['Solution2']))

print(f't_stat: {t_stat}, b_lower: {b_lower}, b_upper: {b_upper}')


In [ ]:
# QQ-Plot for Solution1
plt.figure()
stats.probplot(y1, dist="norm", plot=plt)
plt.title('Normal QQ-Plot for first Solution')
plt.show()

# QQ-Plot for Solution2
plt.figure()
stats.probplot(y2, dist="norm", plot=plt)
plt.title('Normal QQ-Plot for second Solution')
plt.show()

from statsmodels.graphics.gofplots import qqplot_2samples

# QQ-Plot to assess equal variance
plt.figure()
qqplot_2samples(df20['Solution1']- mean_1, df20['Solution2'] - mean_2, xlabel='Solution1', ylabel='Solution2', line='45')
plt.show()

Size of test: the probability, that, given that two means are different, the random variable from the non-central student distribution falls into the acceptance region of the null hypothesis

In [ ]:
from scipy.stats import nct
def power_two_sample_t(n, delta, sigma, alpha):
    df = 2*n - 2
    # noncentrality parameter
    lambda_ = delta / (sigma * np.sqrt(2/n))
    # critical value for two-sided test
    t_crit = t.ppf(1 - alpha/2, df)
    # power calculation using noncentral t CDF
    power = nct.cdf(-t_crit, df, lambda_) + (1 - nct.cdf(t_crit, df, lambda_))
    return power

calc_power = power_two_sample_t(len(df20['Solution1']), abs(mean_1-mean_2), pooled_std, 0.05)
print(f'power of the test: {calc_power}')

To calculate the desired number of observations not using normal approximations, one can use iterative approach:

In [ ]:

# search for minimum n
n = 2
while True:
    pwr = power_two_sample_t(n, 0.3, pooled_std, 0.05)
    if pwr >= 0.9:
        break
    n += 1

print(f"Required sample size per group: {n}")
print(f"Achieved power: {pwr:.4f}")


### Exercise 2.26

The following are the burning times (in minutes) of chemical flares of two different formulations. The design engineers are interested in both the mean and
variance of the burning times.

|Type1|   | Type2 | |
|----|----|----|----|
| 65 | 82 | 64 | 56 |
| 81 | 67 | 71 | 69 |
| 57 | 59 | 83 | 74 |
| 66 | 75 | 59 | 82 |
| 82 | 70 | 65 | 79 |


1. Test the hypothesis that the two variances are equal. Use $\alpha = 0.05$.
2. Using the results of part 1), test the hypothesis that the mean burning
times are equal. Use $\alpha = 0.05$. What is the P-value for this test?
3. Discuss the role of the normality assumption in this problem. Check the
assumption of normality for both types of flares
4. If the mean burning times of the two flares differ by as much as 2 minute, find the power of the test. What sample size would be required to detect an actual difference in mean burning time of 1 minute with a power of at least 0.9?

In [ ]:
# Read the data from the URL
url_26 = "https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex02_26.csv"
df26 = pd.read_csv(url_26, sep=";")

# Display the first few rows of the dataframe
df26.head()

SOLUTION:

Two possibilities of testing equality of variance are presented here, the first one is via classical F-test:

In [ ]:
var_1 = df26['Type1'].var()
var_2 = df26['Type2'].var()

print(f'var_1: {var_1}, var_2: {var_2}')

In [ ]:
F_stat = var_2 / var_1
df1 = len(df26['Type1'])-1
df2 = len(df26['Type2'])-1
p_val = f.sf(F_stat, df1, df2)

print(f'p_val: {p_val}')

The downside of F-test is its reliance on normality of data. The theoretical foundation of F-test relies in the fact that $\frac{\hat{\sigma}^2}{σ} = \frac{1}{σ(n-1)}∑_1^n(X_i - \hat{\mu})^2 ∼ χ^2(n-1)$, which is only true when $X$ is normally distributed and the sample variables $X_i$ are $iid$. Note that central limit theorem does not help in this case.

In [ ]:
mean_1 = df26['Type1'].mean()
mean_2 = df26['Type2'].mean()

pooled_std = np.sqrt(((len(df26['Type1'])-1) * var_1 + (len(df26['Type2'])-1) * var_2) / (len(df26['Type1'])+len(df26['Type2'])-2))

# test statistic
t_stat = (mean_1 - mean_2) / (pooled_std * np.sqrt(1/len(df26['Type1']) + 1/len(df26['Type2'])))
p_value = 2 * min(t.cdf(t_stat, len(df26['Type1'])+len(df26['Type2'])-2), 1 - t.cdf(t_stat, len(df26['Type1'])+len(df26['Type2'])-2))
print(f't_stat: {t_stat}, p_value: {p_value}')


In [ ]:
# let us check the assumption of normality via Shapiro-Wilk test
from scipy.stats import shapiro

stat_1, p_value_1 = shapiro(df26['Type1'])
stat_2, p_value_2 = shapiro(df26['Type2'])

print(f"Shapiro-Wilk test for Type1 \n")
print(f"  W statistic = {stat_1:.4f}")
print(f"  p-value = {p_value_1:.4f}\n")
print(f'Shapiro-Wilk test for Type2 \n')
print(f"  W statistic = {stat_2:.4f}")
print(f"  p-value = {p_value_2:.4f}")



In [ ]:
# to calculate power of the test, we can use the same function as in the previous exercise
calc_power = power_two_sample_t(len(df26['Type1']), 2, pooled_std, 0.05)
print(f'power of the test: {calc_power}')

In [ ]:
# search for minimum n
n = 2
while True:
    pwr = power_two_sample_t(n, 1, pooled_std, 0.05)
    if pwr >= 0.9:
        break
    n += 1

print(f"Required sample size per group: {n}")
print(f"Achieved power: {pwr:.4f}")

### Exercise 2.30

Front housings for cell phones are manufactured in an injection molding process. The time the part is allowed to cool in the mold before removal is thought to influence the occurrence of a particularly troublesome cosmetic defect, flow lines, in the finished housing. After manufacturing, the housings are inspected visually and assigned a score between 1 and 10 based on their appearance, with 10 corresponding to a perfect part and 1 corresponding to a completely defective part. An experiment was conducted using two cool-down times, 10 and 20 seconds, and 20 housings were evaluated at each level of cool-down time. All 40 observations in this experiment were run in random order.


||   |   |10s   || |  |  |20s  |
|---|---|---|---|---|---|---|---|---|
| 1 | 3 | 2 | 6 || 7 | 6 | 8 | 9 |
| 1 | 5 | 3 | 3 || 5 | 5 | 9 | 7 |
| 5 | 2 | 1 | 1 || 5 | 4 | 8 | 6 |
| 5 | 6 | 2 | 8 || 6 | 8 | 4 | 5 |
| 3 | 2 | 5 | 3 || 6 | 8 | 7 | 7 |


* Is there evidence to support the claim that the longer cool-down time
results in fewer appearance defects? Use $\alpha = 0.05$.
* What is the P-value for the test conducted in the previous part?
* Find a 95 percent confidence interval on the difference in means. Provide
a practical interpretation of this interval.
* Compute the power of the test.


In [ ]:
# Read the data from the URL
url_30 = "https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex02_30.csv"
df30 = pd.read_csv(url_30, sep=";")

# Display the first few rows of the dataframe
df30.head()

SOLUTION:

In [ ]:
from scipy.stats import levene
df30.columns = ['10s', '20s']
mean_10 = df30['10s'].mean()
mean_20 = df30['20s'].mean()
std_10 = df30['10s'].std()
std_20 = df30['20s'].std()

# at first, assess the equivalence of variances

stat, p_value_levene = levene(df30['10s'], df30['20s'], center='median')

print("\nLevene’s test for equality of variance")
print(f"  W statistic = {stat:.4f}, p-value = {p_value_levene:.4f}")



In [ ]:
# the test showed that there is not enough evidence that the variances are different, so lets use the assumption of equal variances
pooled_std = np.sqrt(((len(df30['10s'])-1) * std_10**2 + (len(df30['20s'])-1) * std_20**2) / (len(df30['10s'])+len(df30['20s'])-2))

# test statistic (remember - the higher the mean, the less appearance defects so we need to check whether the statistic is sufficiently above zero)
t_stat = (mean_20 - mean_10) / (pooled_std * np.sqrt(1/len(df30['10s']) + 1/len(df30['20s'])))
# p-value
p_value = 1-t.cdf(t_stat, len(df30['10s'])+len(df30['20s'])-2)
print(f't_stat: {t_stat}, p_value: {p_value}')

In [ ]:
# now find the 0.95 confidence interval:

b_lower = mean_20 - mean_10 - (t.ppf(0.975, len(df30['10s'])+len(df30['20s'])-2) * pooled_std * np.sqrt(1/len(df30['10s']) + 1/len(df30['20s'])))
b_upper = mean_20 - mean_10 + (t.ppf(0.975, len(df30['10s'])+len(df30['20s'])-2) * pooled_std*np.sqrt(1/len(df30['10s']) + 1/len(df30['20s'])))

print(f'b_lower: {b_lower}, b_upper: {b_upper}')

**95 percent confidence interval**: true difference between means lies within borders of this interval with probability of 0.95

In [ ]:
power_two_sample_t(len(df30['10s']), abs(mean_20-mean_10), pooled_std, 0.05)